In [4]:
# !pip install transformers

In [113]:
import pandas as pd
import numpy as np
# import matplotlib.pyplot as plt
# import seaborn as sns
import warnings
import os

import torch
from transformers import BertTokenizer, BertModel
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.preprocessing import StandardScaler
# os.chdir("/home/nikosili/projects/datatalks_zoomcamp/Machine_Learning_Engineering_ZooCamp/QA_challenge")
# 
warnings.filterwarnings('ignore')
# sns.set_theme()
%matplotlib inline

In [6]:
train_a = pd.read_csv('data/train_answers.csv')
train_q = pd.read_csv('data/train_questions.csv')
test_q = pd.read_csv('data/test_questions.csv')
test_a = pd.read_csv('data/test_answers.csv')

### Train set

In [7]:
print(train_q.iloc[0][1],'\n\n',train_a.iloc[0][1])


For categorical target set, where the distribution is imbalanced (for example, 90/10) what approach should be used? 

 Alexey
Should we use something non-standard there or can we just go with the usual things we learned in the course?
Hamed
You just need to test different strategies. Something I noticed – if you have so many parse subclasses in your categorical [inaudible], you should be careful about using one-hot encoding. You might say you can use ordinal encoding, if your data in nature had some order. It will be useful. In my particular data, I couldn't have domain knowledge. I didn't know what the subclasses were, so I couldn't decide which strategy I should choose. But if you have the domain knowledge, that’s the key here, I think.


In [8]:
train_q.head()

,question_id,question,course,year,candidate_answers,answer_id
0,79062,"For categorical target set, where the distribu...",Machine Learning Zoomcamp,2021,"156400,754877,105368,643810,912439",156400
1,468946,Is there anything that we are not allowed to u...,Machine Learning Zoomcamp,2021,"641330,634887,912439,425941,642829",634887
2,968800,I have been catching up and have been doing ho...,Data Engineering Zoomcamp,2022,"954016,167856,75919,36798,838013",954016
3,688404,Could you please explain what code we should l...,Data Engineering Zoomcamp,2022,"198661,629898,686577,3699,141765",3699
4,63921,Is it just me or does the model have really ba...,Machine Learning Zoomcamp,2021,"754877,604487,912439,858915,425941",858915


In [9]:
train_a.head()

,answer_id,answer,course,year,attachments_files
0,156400,Alexey\nShould we use something non-standard t...,Machine Learning Zoomcamp,2021,NaN
1,634887,"No, I don't think there is anything you cannot...",Machine Learning Zoomcamp,2021,NaN
2,954016,"Alexey\nYes, you will be. You can submit the p...",Data Engineering Zoomcamp,2022,NaN
3,3699,Alexey\nI think the question refers to the hom...,Data Engineering Zoomcamp,2022,NaN
4,858915,"Dmitry\nIt's fine, because this is the showcas...",Machine Learning Zoomcamp,2021,NaN


In [10]:
temp_1 = train_q.groupby(['question_id', 'answer_id']).size().reset_index().rename(columns={0:'rec_count'})
temp_1.groupby('question_id').size().reset_index().rename(columns={0:'rec_count'}).sort_values('rec_count', ascending=False)

,question_id,rec_count
279,647840,2
0,26,1
261,605034,1
270,622805,1
269,619694,1
...,...,...
129,282338,1
128,278678,1
127,274935,1
126,273744,1


In [11]:
temp_1.groupby('answer_id').size().reset_index().rename(columns={0:'rec_count'}).sort_values('rec_count', ascending=False)

,answer_id,rec_count
310,774098,2
0,3699,1
270,673096,1
269,668438,1
268,666820,1
...,...,...
129,325935,1
128,323402,1
127,322444,1
126,317865,1


In [12]:
# print(train_q[train_q['answer_id'] == 774098].answr)
print(train_q[train_q['answer_id'] == 774098].question.iloc[0])
print(train_q[train_q['answer_id'] == 774098].question.iloc[1],'\n')
print(train_a[train_a['answer_id'] == 774098].answer)

Can we use PyTorch instead of TensorFlow?
For rating data (number of stars) can we use a linear regression model? 

261    Yes, you can.
358    Yes, you can.
Name: answer, dtype: object


In [13]:
print(train_q.shape)
temp = train_q.drop_duplicates(subset=['answer_id'], keep='first')
print(temp.shape)
print(temp[temp['answer_id'] == 774098].question.iloc[0])

(397, 6)
(396, 6)
Can we use PyTorch instead of TensorFlow?


In [14]:
train_q[train_q['question_id'] == 647840].question

72     How many submissions were there this week?
159    How many submissions were there this week?
Name: question, dtype: object

In [15]:
list(train_q[train_q['question_id'] == 647840].question.index)

[72, 159]

In [16]:
print(train_a[(train_a['answer_id'] == 276086) | (train_a['answer_id'] == 964992)].answer.iloc[0],'\n')
print(train_a[(train_a['answer_id'] == 276086) | (train_a['answer_id'] == 964992)].answer.iloc[1])

I think around 200? 204, I think. Something like this. 

Alexey
246, which is 150 less than last week. Last week, it was 404. I don’t know if this homework was maybe a bit more difficult. But the next one is also fun. I think if you liked the homework we prepared for this week, the next one will also be fun. Right, Dmitry? 
Dmitry
Yeah, for sure.


In [17]:
print(temp.shape)
temp = temp.drop(list(train_q[train_q['question_id'] == 647840].question.index))
print(temp.shape)



(396, 6)
(394, 6)


### Test set

In [18]:
test_a[test_a.duplicated()]

,answer_id,answer,course,year,attachments_files
73,774098,"Yes, you can.",Data Engineering Zoomcamp,2023,NaN


In [19]:
train_q.head()

,question_id,question,course,year,candidate_answers,answer_id
0,79062,"For categorical target set, where the distribu...",Machine Learning Zoomcamp,2021,"156400,754877,105368,643810,912439",156400
1,468946,Is there anything that we are not allowed to u...,Machine Learning Zoomcamp,2021,"641330,634887,912439,425941,642829",634887
2,968800,I have been catching up and have been doing ho...,Data Engineering Zoomcamp,2022,"954016,167856,75919,36798,838013",954016
3,688404,Could you please explain what code we should l...,Data Engineering Zoomcamp,2022,"198661,629898,686577,3699,141765",3699
4,63921,Is it just me or does the model have really ba...,Machine Learning Zoomcamp,2021,"754877,604487,912439,858915,425941",858915


In [20]:
can_ans = train_q.iloc[0]['candidate_answers'].split(',')

In [23]:
# for idx in can_ans:
#     print(idx,'\n', train_a[train_a['answer_id']==int(idx)].iloc[0].answer)

In [25]:
# text=''
# for idx in can_ans:
#     temp_text=train_a[train_a['answer_id']==int(idx)].iloc[0].answer
#     text+=temp_text
# text

In [26]:
# Let's marge the train questions and answers in one dataframe
train_merged_df = pd.merge(
    temp, train_a, on='answer_id', how='inner', suffixes=('_question', '_answer')
)
train_merged_df.head()

,question_id,question,course_question,year_question,candidate_answers,answer_id,answer,course_answer,year_answer,attachments_files
0,79062,"For categorical target set, where the distribu...",Machine Learning Zoomcamp,2021,"156400,754877,105368,643810,912439",156400,Alexey\nShould we use something non-standard t...,Machine Learning Zoomcamp,2021,NaN
1,468946,Is there anything that we are not allowed to u...,Machine Learning Zoomcamp,2021,"641330,634887,912439,425941,642829",634887,"No, I don't think there is anything you cannot...",Machine Learning Zoomcamp,2021,NaN
2,968800,I have been catching up and have been doing ho...,Data Engineering Zoomcamp,2022,"954016,167856,75919,36798,838013",954016,"Alexey\nYes, you will be. You can submit the p...",Data Engineering Zoomcamp,2022,NaN
3,688404,Could you please explain what code we should l...,Data Engineering Zoomcamp,2022,"198661,629898,686577,3699,141765",3699,Alexey\nI think the question refers to the hom...,Data Engineering Zoomcamp,2022,NaN
4,63921,Is it just me or does the model have really ba...,Machine Learning Zoomcamp,2021,"754877,604487,912439,858915,425941",858915,"Dmitry\nIt's fine, because this is the showcas...",Machine Learning Zoomcamp,2021,NaN


In [50]:
df = pd.DataFrame(columns=['context'])
temp_2 = []
for i in train_merged_df.candidate_answers:
    temp_2.append(i.split(','))


In [53]:
df_temp.context.iloc[0]
len(temp_2)

395

In [54]:
for i in temp_2:
    text = ''
    for j in i:
        text_temp=train_a[train_a['answer_id']==int(j)].iloc[0].answer
        text+=text_temp
    con = {'context':text}
    df_temp = pd.DataFrame([con])
    # df = df.append(df_temp, ignore_index=True)
    df = pd.concat([df, df_temp],ignore_index=True)

In [55]:
df

,context
0,Alexey\nShould we use something non-standard t...
1,"44. Two people submitted twice. So actually, i..."
2,"Alexey\nYes, you will be. You can submit the p..."
3,"Alexey\nIt's not over for you, because we only..."
4,I don't know if you're referring to this. [ima...
...,...
390,I took this course a while ago. Back then it w...
391,You mean another iteration of the course? Yes....
392,Ankush\nIf you're talking specifically about i...
393,If two people want to work on the same dataset...


In [56]:
train = pd.concat([train_merged_df, df], axis=1)

In [64]:
train_a

,answer_id,answer,course,year,attachments_files
0,156400,Alexey\nShould we use something non-standard t...,Machine Learning Zoomcamp,2021,NaN
1,634887,"No, I don't think there is anything you cannot...",Machine Learning Zoomcamp,2021,NaN
2,954016,"Alexey\nYes, you will be. You can submit the p...",Data Engineering Zoomcamp,2022,NaN
3,3699,Alexey\nI think the question refers to the hom...,Data Engineering Zoomcamp,2022,NaN
4,858915,"Dmitry\nIt's fine, because this is the showcas...",Machine Learning Zoomcamp,2021,NaN
...,...,...,...,...,...
392,831391,"Yes, it can. It's really dataset dependent. Fo...",Machine Learning Zoomcamp,2021,NaN
393,651754,"Let's say I do “import numpy as np” and then, ...",Machine Learning Zoomcamp,2021,NaN
394,733226,Alexey\nSplunk – I don’t know. It's not a data...,Data Engineering Zoomcamp,2022,NaN
395,422297,"Yes, it was not mentioned. But what was mentio...",Machine Learning Zoomcamp,2021,NaN


In [91]:
# Use GPU if available
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Available device: {device}")

Available device: cpu


In [107]:
# Getting pre-trained BERT model
model_name = 'bert-base-uncased'
tokenizer = BertTokenizer.from_pretrained(model_name)
model = BertModel.from_pretrained(model_name)
model.to(device) # Move model to GPU

model.safetensors: 100%|██████████| 440M/440M [01:31<00:00, 4.83MB/s] 


BertModel(
  (embeddings): BertEmbeddings(
    (word_embeddings): Embedding(30522, 768, padding_idx=0)
    (position_embeddings): Embedding(512, 768)
    (token_type_embeddings): Embedding(2, 768)
    (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
    (dropout): Dropout(p=0.1, inplace=False)
  )
  (encoder): BertEncoder(
    (layer): ModuleList(
      (0-11): 12 x BertLayer(
        (attention): BertAttention(
          (self): BertSelfAttention(
            (query): Linear(in_features=768, out_features=768, bias=True)
            (key): Linear(in_features=768, out_features=768, bias=True)
            (value): Linear(in_features=768, out_features=768, bias=True)
            (dropout): Dropout(p=0.1, inplace=False)
          )
          (output): BertSelfOutput(
            (dense): Linear(in_features=768, out_features=768, bias=True)
            (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
            (dropout): Dropout(p=0.1, inplace=False)
  

In [108]:
def get_bert_embeddings(text):
    """
    Function for getting text embeddings using BERT
    Returns one embedding as an average of the words embeddings of the text 
    """
    tokens = tokenizer(text, return_tensors='pt', truncation=True, padding=True)
    tokens = {key: value.to(device) for key, value in tokens.items()}
    with torch.no_grad():
        outputs = model(**tokens)
    return outputs['last_hidden_state'][0].mean(dim=0).cpu().numpy()

In [106]:
# tokenized_sentence = tokenizer.encode(test_sentence, padding=True, truncation=True,max_length=50, add_special_tokens = True)

In [109]:
# Get questions and answers embeddings for the train part
train_question_embeddings = train['question'].apply(get_bert_embeddings)
train_answer_embeddings = train['answer'].apply(get_bert_embeddings)

In [114]:
# Normalization
scaler = StandardScaler()
train_question_embeddings_standardized = scaler.fit_transform(np.array(train_question_embeddings.tolist()))
train_answer_embeddings_standardized = scaler.transform(np.array(train_answer_embeddings.tolist()))

In [121]:
def get_predictions(df_questions, df_answers):
    """
    Function that finds the best answer to each question according to their similarity.
    """
    predicted_answers = []
    predicted_answer_ids = []

    for index, row in df_questions.iterrows():
        question_text = row['question']
        candidate_answer_ids = [int(answer_id) for answer_id in row['candidate_answers'].split(",")]

        # Getting questions embeddings
        question_embedding = get_bert_embeddings(question_text)
        question_embedding_standardized = scaler.transform(question_embedding.reshape(1, -1))

        # Getting answer candidate embeddings
        candidate_answers_df = df_answers[df_answers['answer_id'].isin(candidate_answer_ids)]
        candidate_answer_embeddings = candidate_answers_df['answer'].apply(get_bert_embeddings)
        candidate_answer_embeddings_standardized = scaler.transform(np.array(candidate_answer_embeddings.tolist()))
        standardized = scaler.transform(np.array(candidate_answer_embeddings.tolist()))

        # Calculating similarity between question and answers embeddings
        similarities = cosine_similarity(question_embedding_standardized, candidate_answer_embeddings_standardized).flatten()

        # Taking index of the best answer candidate
        best_answer_index = similarities.argmax()

        predicted_answer = candidate_answers_df.iloc[best_answer_index]['answer']
        predicted_answer_id = candidate_answers_df.iloc[best_answer_index]['answer_id']
        predicted_answers.append(predicted_answer)
        predicted_answer_ids.append(predicted_answer_id)
        
    return predicted_answer_ids, predicted_answers

In [122]:
train_predictions_df = pd.DataFrame({
    'question_id': train['question_id'],     
    'question': train['question'],
    'candidate_answers': train['candidate_answers'],
    'answer_id': train['answer_id'],
})

train_predictions_df['predicted_answer_id'], train_predictions_df['predicted_answer'] = \
    get_predictions(train_q, train_a)

train_predictions_df.head()

KeyboardInterrupt: 

In [ ]:
train_predictions_df.head()

# Accuracy calculation
accuracy = (train_predictions_df['predicted_answer'] == train_answers_df['answer']).mean()
print(f'Accuracy: {accuracy:.4f}')

# Now we can use the same approach to get answers for the test part.
test_questions_df = test_questions_df.drop_duplicates(subset='question_id')
test_questions_df.shape

# Creating the dataframe for the test part
test_predictions_df = pd.DataFrame({
    'question_id': test_questions_df['question_id'], 
})

test_predictions_df['predicted_answer_id'], test_predictions_df['predicted_answer'] = \
    get_predictions(test_questions_df, test_answers_df)

test_predictions_df.head()

test_predictions_df[['question_id', 'predicted_answer_id']].to_csv('BERT_sample_submission.csv', index=False)